#Домашнє завдання до тем apply(), groupby(), pivot_table()

В цьому домашньому завданні продовжуємо працювати з набором даних `supermarket_sales.csv`.

0. Імпортуйте бібліотеку pandas та зчитайте дані у змінну `df` типу `pandas.DataFrame`.

In [59]:
import pandas as pd
df = pd.read_csv("../data/supermarket_sales.csv")
type(df)


pandas.core.frame.DataFrame

1. Дослідимо, який філіал супермаркету ('Branch') є найприбутковішим. Для цього знайдіть сумарний прибуток за кожним філіалом і виявіть, який філіал має найвищий.

In [60]:
df.groupby('branch')['revenue'].sum().sort_values(ascending = False)

branch
C    110568.7065
A    106200.3705
B    106197.6720
Name: revenue, dtype: float64

2. В якому місті знайходиться філіал з найвищим прибутком? Може в тому місці нам розмітисти ще один магазин.  
Знайдіть відповідь за допомогою функціоналу Pandas.

In [61]:
df.groupby(['city', 'branch'])['revenue'].sum().sort_values(ascending = False)

city       branch
Naypyitaw  C         110568.7065
Yangon     A         106200.3705
Mandalay   B         106197.6720
Name: revenue, dtype: float64

3.1. Створіть зводну таблицю, яка покаже, скільки покупок (інвойсів) було зроблено в кожній з філій (`Branch`) за різними категоріями товарів. Запишіть таблицю в змінну `invoices_by_category` і виведіть змінну на екран.
Ця таблиця допоможе проаналізувати, в якій філії купують найбільше товарів кожної з категорій.

In [62]:
df.columns

Index(['invoice_id', 'branch', 'city', 'customer_type', 'gender_customer',
       'product_line', 'unit_cost', 'quantity', '5pct_markup', 'revenue',
       'date', 'time', 'payment_method', 'cogs', 'gm_pct', 'gross_income',
       'rating'],
      dtype='object')

In [63]:
invoices_by_category = pd.pivot_table(
    df, 
    index = 'branch',
    columns = 'product_line',
    values = 'invoice_id',
    aggfunc = 'count'
)
invoices_by_category 


product_line,Electronic accessories,Fashion accessories,Food and beverages,Health and beauty,Home and lifestyle,Sports and travel
branch,,,,,,
A,60,51,58,47,65,59
B,55,62,50,53,50,62
C,55,65,66,52,45,45


Очікуваний результат:

![](https://drive.google.com/uc?export=view&id=1rueAdko6S3UxIHGtojetTxlES-EyM6Yb)

3.2. Викристовуючи змінну `invoices_by_category` дайте відповідь програмно (тобто значення треба не просто знайти очима, а вивести за допомогою коду), в якому філіалі магазину (`Branch`) найбільше інвойсів із покупкою товарів категорії "Електронні аксесуари" (`Electronic accessories`)?


In [64]:
branch = invoices_by_category['Electronic accessories'].idxmax()
count = invoices_by_category['Electronic accessories'].max()
branch, count

('A', np.int64(60))

4-6. **Творче завдання на розвиток аналітичного мислення**

Крок 1. Сформулюйте ТРИ питання (гіпотези) до наявних даних, які допомогли б вам зрозуміти, які користувачі що, де та коли найбільше/найменше купують, аби дати на основі цих гіпотез рекомендації бізнесу. Звісно питання мають бути не тими, на які ми вже відповіли в завданнях модулю.

Крок 2. Знайдіть відповіді на свої питання з допомогою функціоналу pandas.

Крок 3. Напишіть, як відповідь на це питання може бути використана для прийняття бізнес рішень.   
   
 Питання можуть бути будь-якої складності, але їх має бути 3. Кожне питання оцінюється як 1 завдання. Без виконання цього завдання ДЗ не приймається. Якщо є питання щодо виконання - пишіть у чат 🙌


Гіпотеза 1 
Чи відрізняється частота покупок залежно від способу оплати?

In [65]:
payment_method = df.payment_method.value_counts().sort_values(ascending = False)
payment_method

payment_method
Ewallet        345
Cash           344
Credit card    311
Name: count, dtype: int64

Кількість покупок за різними способами оплати є приблизно однаковою, тому сам по собі спосіб оплати не має істотного впливу на частоту покупок. Доцільно додатково проаналізувати середній чек та маржу за кожним методом оплати, щоб оцінити їхню фінансову ефективність для бізнесу.

In [66]:
mean_revenue_by_payment = df.groupby('payment_method')['revenue'].mean().round(2)
mean_revenue_by_payment

payment_method
Cash           326.18
Credit card    324.01
Ewallet        318.82
Name: revenue, dtype: float64

In [67]:
mean_gross_income_by_payment = df.groupby('payment_method')['gross_income'].mean().round(2)
mean_gross_income_by_payment

payment_method
Cash           15.53
Credit card    15.43
Ewallet        15.18
Name: gross_income, dtype: float64

Аналіз показав, що спосіб оплати E-wallet використовується найчастіше, однак середній чек та середній прибуток для цього методу не є найвищими. Найбільш вигідні для бізнесу покупки в середньому здійснюються при оплаті готівкою, що може свідчити про більший розмір кошика при такому способі оплати.
Доцільно зберігати зручність готівкового методу оплати, а також стимулювати збільшення середнього чеку при безготіковому способі оплати, залучаючи акції чи бонуси.

На даному моменті відчула, що має бути поставлена задача/проблема/питання від бізнесу до аналітика, що від цього відштовхувтись та фомулювати гіпотези. Тож припустимо, що поставдена задача:
"Дослідити методи оплати на впливосвісь на прибуток, та виявити слабкі місця, де можна покращити фінансовий результат.

Гіпотеза 1. «Чи відрізняються частота покупок та прибутковість залежно від методу оплати?»

In [68]:
df.groupby('payment_method').agg(
    purchases = ('invoice_id', 'count'),
    arg_revenue = ('revenue', 'mean'),
    arg_gross_income = ('gross_income', 'mean'),
    total_gross_income = ('gross_income', 'sum')
).round(2)

,purchases,arg_revenue,arg_gross_income,total_gross_income
payment_method,,,,
Cash,344,326.18,15.53,5343.17
Credit card,311,324.01,15.43,4798.43
Ewallet,345,318.82,15.18,5237.77


По зведеній таблиці бачимо, що кількість покупок за різними методами оплати є подібною, однак фінансові показники відрізняються.

Спосіб оплати E-wallet використовується найчастіше, проте він має найнижчий середній чек та середній прибуток з однієї покупки. Водночас оплата готівкою  характеризується найвищим середнім чеком та середнім прибутком, що робить цей метод найбільш вигідним для бізнесу з точки зору прибутковості.

Метод оплати Credit card займає проміжне положення як за частотою використання, так і за рівнем прибутковості.

Таким чином, можна зробити висновок, що в даному випадку частота використання способу оплати не означає його фінансовою ефективністю. Для підвищення загальної прибутковості бізнесу доцільно зберігати зручність безготівкових методів оплати, водночас стимулюючи збільшення середнього чеку для покупок з використанням E-wallet.

Гіпотеза 2. «Чи означає більший середній чек більшу маржу для бізнесу в межах кожного методу оплати?»

In [69]:
df.groupby('payment_method').agg(
    agr_revenue = ('revenue', 'mean'),
    agr_gross_income = ('gross_income', 'mean'),
    agr_margin_pct = ('gm_pct', 'mean')
).round(2)

,agr_revenue,agr_gross_income,agr_margin_pct
payment_method,,,
Cash,326.18,15.53,4.76
Credit card,324.01,15.43,4.76
Ewallet,318.82,15.18,4.76


Гіпотеза не підтвердилась. Аналіз показав, що маржинальність продажів є сталою для всіх методів оплати, що свідчить про використання фіксованої націнки незалежно від способу здійснення покупки. Різниця в прибутковості між методами оплати зумовлена не зміною маржі, а відмінностями в середньому чеку та кількості покупок. Тобто доцільно стимулювати прибуток за рахунок збільшення кількості покупок або середнього чеку. (Так як метод E-wallet має високий (вищий) трафік, то варто зосередитись на ньому та стимулювати збільшення середнього чеку.)

In [70]:
Гіпотеза 3. Які категорії товарів найбільше та найменше продаються при оплаті через E-wallet?

Object `wallet` not found.


In [71]:
ewallet_df =df[ df['payment_method'] == 'Ewallet']

In [72]:
ewallet_df['product_line'].value_counts(normalize = True).round(2)*100

product_line
Fashion accessories       19.0
Home and lifestyle        19.0
Food and beverages        16.0
Sports and travel         16.0
Health and beauty         15.0
Electronic accessories    15.0
Name: proportion, dtype: float64

Розподіл досить рівномірний, без різких перекосів.
Користувачі E-wallet купують різні категорії.

In [73]:
ewallet_df.groupby('product_line').agg(
    purchases = ('invoice_id', 'count'),
    agr_revenue = ('revenue', 'mean'),
    arg_gross_income = ('gross_income', 'mean') 
).sort_values('arg_gross_income', ascending = False).round(2)

,purchases,agr_revenue,arg_gross_income
product_line,,,
Electronic accessories,53,343.01,16.33
Sports and travel,54,341.35,16.25
Home and lifestyle,64,332.65,15.84
Health and beauty,53,302.55,14.41
Food and beverages,56,298.19,14.20
Fashion accessories,65,297.80,14.18


Найбільш вигідні категорії через E-wallet: 'Electronic accessories' — найвищий середній чек і прибуток, але купуються рідше, а також 'Sports and travel' та 'Home and lifestyle'. 
Найменш вигідні категорії через E-wallet: 'Fashion accessories', 'Food and beverages' - купуються найчастіше, але найменш прибуткові.
Аналіз продажів при оплаті через E-wallet показав, що найбільш часто купуються категорії 'Fashion accessories' та 'Home and lifestyle', однак вони не є найбільш прибутковими з точки зору середнього чеку та прибутку. Водночас категорії 'Electronic accessories' та 'Sports and travel' характеризуються вищим середнім чеком і прибутковістю, проте купуються дещо рідше. Це свідчить про потенціал для збільшення середнього чеку шляхом стимулювання продажів більш прибуткових категорій серед користувачів E-wallet.